## DHK Airdrop Script

In [1]:
# These are parameters used in the program below
from datetime import datetime, timezone
from dotenv import load_dotenv
from os import environ

load_dotenv()

DHK_DISTRIBUTED = 100000
# 2024-07-29 12:00
PRICE_REF_DATE = datetime(2024, 7, 29, 12, tzinfo=timezone.utc)

TOKENS = [
    { "token": "AKT", "qty": 189628 },
    { "token": "ATOM", "qty": 592191 },
    { "token": "DSM", "qty": 3967661, "price": 0.002867 },
    { "token": "EVMOS", "qty": 248824 },
]

APIS = {
    "cryptocompare": {
        "endpoint": "https://min-api.cryptocompare.com/data/pricehistorical",
        "apikey": environ.get('CRYPTOCOMPARE_APIKEY')
    }
}

In [2]:
# Program to retrieve necessary token information for export
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import math
import random
import pandas as pd

# API doc: https://min-api.cryptocompare.com/documentation?key=Historical&cat=dataPriceHistorical
def fetch_price(from_symbol, to_symbol='USD', date=PRICE_REF_DATE):
    endpoint, apikey = APIS["cryptocompare"]["endpoint"], APIS["cryptocompare"]["apikey"]
    parameters = {
        'fsym':from_symbol,
        'tsyms': to_symbol,
        'calculationType':'MidHighLow',
        'ts': date.timestamp()
    }
    headers = {
        'Accepts': 'application/json',
        'authorization': f"Apikey {apikey}",
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(endpoint, params=parameters)
        data = json.loads(response.text)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        raise Exception(f"fetch_price connection error: {e}")

    if(from_symbol not in data):
        raise Exception(f"unable to fetch price, returning: {data}")
    
    return data[from_symbol][to_symbol]    

def fetch_inflation(symbol, date=PRICE_REF_DATE):
    # TODO: fetching inflation rate
    return random.uniform(0, 40)/100

def get_main_table():
    columns = ["token", "price", "staked_amt", "staked_val", "inflation", "reward", "distributed", "distributed_pc"]
    lst = []
    
    for t in TOKENS:
        token, staked_amt = t["token"], t["qty"]
        token_price = t["price"] if "price" in t else round(fetch_price(token), 5)
        staked_val = round(token_price * staked_amt, 2)
        inflation = round(fetch_inflation(token), 4)
        reward = round(inflation * staked_val, 2)
        lst.append([token, token_price, staked_amt, staked_val, inflation, reward, 0, 0])

    mt = pd.DataFrame(lst, columns=columns)
    
    # Append a total row at the end of the table
    ttl_staked_val = mt["staked_val"].sum()
    ttl_reward = mt["reward"].sum()
    ttl = pd.Series({"token": "TOTAL", "staked_val": ttl_staked_val, "reward": ttl_reward})
    mt = pd.concat([mt, ttl.to_frame().T], ignore_index=True)

    # Calculate DHK distribution and distribution percent
    for idx, row in mt.iterrows():
        mt.at[idx, "distributed"] = math.floor(row["reward"] / ttl_reward * DHK_DISTRIBUTED)
        mt.at[idx, "distributed_pc"] = round((row["reward"] / ttl_reward) * 100, 2)
    
    return mt

get_main_table()

,token,price,staked_amt,staked_val,inflation,reward,distributed,distributed_pc
0,AKT,3.225500,189628.0,611645.11,0.0481,29420.13,4193.0,4.19
1,ATOM,6.238000,592191.0,3694087.46,0.1810,668629.83,95308.0,95.31
2,DSM,0.002867,3967661.0,11375.28,0.0667,758.73,108.0,0.11
3,EVMOS,0.029390,248824.0,7312.94,0.3741,2735.77,389.0,0.39
4,TOTAL,NaN,NaN,4324420.79,NaN,701544.46,100000.0,100.00
